# Stars: *

### Day 10: Pipe Maze

#### Part 1

In [1]:
def get_data(file_name):
    # read all lines in text file
    with open(file_name, "r") as file:
        lines = file.readlines() 

    # remove newline text from each line
    lines = [line.rstrip("\n").split(" ") for line in lines]

    # get metadata
    n_rows = len(lines)
    n_cols = len(lines[0][0])
    print(f"Shape: {n_cols} columns, {n_rows} rows")
    return lines

lines = get_data("test_input.txt")
lines

Shape: 5 columns, 5 rows


[['.....'], ['.S-7.'], ['.|.|.'], ['.L-J.'], ['.....']]

In [2]:
def display_grid(grid):
    for row in grid:
        print("".join(row))

In [3]:
len(lines)

5

In [4]:
def generate_grid(lines, print_grid=False):
    grid = [[' ' for _ in range(len(lines))] for _ in range(len(lines))]
    for i, row in enumerate(lines):
        for j, element in enumerate(row[0]):
            grid[i][j] = element
    if print_grid==True:
        display_grid(grid)
    return grid

In [5]:
grid = generate_grid(lines, print_grid=False)

In [6]:
def find_start(grid):
    for i, row in enumerate(grid):
        for j, value in enumerate(row):
            if value == "S":
                start_pos = (i, j)
                return start_pos

# example
start_pos = find_start(grid)
print(start_pos)

(1, 1)


In [7]:
offsets_dic = {
    "down": [1, 0],
    "up": [-1, 0],
    "right": [0, 1],
    "left": [0, -1]
}

possible_values_dic = {
    "down": ["|", "L", "J"],
    "up": ["|", "F", "7"],
    "right": ["-", "J", "7"],
    "left": ["-", "L", "F"]
}

In [8]:
def find_valid_next_steps(pos, grid, visited):

    valid_next_steps = []

    for dir, offset in offsets_dic.items():
        
        # get offset position
        pos_row = pos[0]
        pos_col = pos[1]
        offset_row = pos_row + offset[0]
        offset_col = pos_col + offset[1]
        offset_pos = [offset_row, offset_col]

        # exclude visited paths
        if offset_pos not in visited:
        
            # get offset position's value
            offset_val = grid[offset_row][offset_col]
            possible_values = possible_values_dic[dir]
            
            # check if valid
            if offset_val in possible_values:
                valid_next_steps.append(offset_pos)

    return valid_next_steps

# example
visited = [[1, 1], [1, 2]] #[find_start(grid)]
pos = [1, 3]
find_valid_next_steps(pos, grid, visited)

[[2, 3]]

In [9]:
from collections import deque

In [10]:
lines = get_data("test_input.txt")
grid = generate_grid(lines, print_grid=False)

start_pos = next_position = find_start(grid)

print("Start", next_position)
visited = [next_position]

while True:
    try: 
        next_position = find_valid_next_steps(next_position, grid, visited)
        next_position = next_position[0]
        visited.append(next_position)
    except IndexError:
        break

print(visited)

for pos in visited: 
    grid[pos[0]][pos[1]]="#"
display_grid(grid)

Shape: 5 columns, 5 rows
Start (1, 1)
[(1, 1), [2, 1], [3, 1], [3, 2], [3, 3], [2, 3], [1, 3], [1, 2]]
.....
.###.
.#.#.
.###.
.....


In [11]:
import sys
sys.setrecursionlimit(1000000000)

In [12]:
def find_paths_recursive(grid, current_path=[(0,0)], solutions=[]):

    n = len(grid)
    last_cell = current_path[-1]

    dirs = {
        "down": [1, 0],
        "up": [-1, 0],
        "right": [0, 1],
        "left": [0, -1]
    }
    possible_values_dic = {
        "down": ["|", "L", "J"],
        "up": ["|", "F", "7"],
        "right": ["-", "J", "7"],
        "left": ["-", "L", "F"]
    }
    
    for dir, offset in dirs.items():
        x, y = offset
        new_i = last_cell[0] + x
        new_j = last_cell[1] + y
        new_cell = (new_i, new_j)
        # print("Try", new_cell)

        # get possible values
        possible_values = possible_values_dic[dir]
        
        # Check if new cell is in grid
        if 0 > new_i or new_i >= n or 0 > new_j or new_j >= n:
            # status = ">>> out of index"
            continue

        # Check if new cell is already in path
        if new_cell in current_path[1:]:
            # status = ">>> already in path"
            continue

        # get new value
        new_val = grid[new_i][new_j]

        # Check if valid cell
        if new_val not in possible_values:
            # status = f">>> Non-valid value: {new_val}"
            continue

        # print(">>> Valid")

        # Add cell to current path
        current_path_copy = current_path.copy()
        current_path_copy.append(new_cell)
        # print("Path:", current_path_copy)

        if new_i==n-1 and new_j == n-1:
            solutions.append(current_path_copy)

        solutions.append(current_path_copy)

        # Create new current_path array for every direction
        find_paths_recursive(grid, current_path_copy, solutions)
        
    return solutions

def compute_cell_values(grid1, solutions):
    path_values = []
    for solution in solutions:
        solution_values = []
        for cell in solution:
            solution_values.append(grid1[cell[0]][cell[1]])
        path_values.append(solution_values)
    return path_values

lines = get_data("input.txt")
grid = generate_grid(lines, print_grid=False)

start_pos = next_position = find_start(grid)

solutions = find_paths_recursive(grid, current_path=[start_pos])
path_values = compute_cell_values(grid, solutions)

# print('Solutions:')
# print(solutions)
# print('Values:')
# print(path_values)


Shape: 140 columns, 140 rows


In [16]:
print(len(solutions))

881004


In [17]:
from tqdm import tqdm

In [18]:
steps_dic = {start_pos: 0}
for path in tqdm(solutions):
    start = path[0]
    end = path[-1]
    steps = len(path) - 1
    if end in steps_dic.keys():
        if steps < steps_dic[end]:
            steps_dic[end] = steps
    else:
        steps_dic[end] = steps
# steps_dic

100%|██████████| 881004/881004 [00:00<00:00, 1096659.18it/s]


In [19]:
lines = get_data("input.txt")
grid = generate_grid(lines, print_grid=False)
for pos in steps_dic.keys():
    grid[pos[0]][pos[1]]="#"
display_grid(grid)

Shape: 140 columns, 140 rows
.7F77F-J-F.LF-|-LL.J####.#####L-L.#LFFJ7-7777-|7-F7.LFF--7F.LJ#################L-7###F--.J#..###LJ#..|7.F|--7####|7F--J---JF-LL-L7-|.|-LJ7.F
FL7J-7FLLL..|||.####.#.########|-J##LJ-7.L7L--7-7.|7F-JFFJJ####################L-J###|-J.############L7.FJJLF-7##FLJ..LF7-L7JL|FL|-77.F-|.F|
7-7FF.||..FJ-|JJ###############F7.####LJ77L7J|F-7JJL-J7-|J######.####################F7J#############LJ.|7JF-JJ##.L|.-7JL77|-7LL.7JF77-.L-F.
7JLJJ|JFLFJ.L|###############.#L|#######LJ.|-|.FJ.-|L-JL.#####.####.################.LL77####.########L--7.JF|.###.L-JJ.L-F-7..|LJ-JF|7-7.L-
LJF|.|L---77-J7####.#######.####.###..###.J7-|F7FJLJ########.########################.L77#####.#######.##L-J-J.###FF.7-7LL|-LLF7|-LFJLF7LFL7
L-LL-JFL|.|-7L7############################L-L-J|J##################.###############.J|.J##########.#####F|..####.|-FJF-7F--FJ||F.F77-||-7FJ
L7-L.|7F|FL--F.#################################.L..F7.#.############################LJ#######.##########F7.######.|L7LF-7J.L

In [26]:
max_steps = max(steps_dic.values())
print(max_steps)

6812


#### Part 2

In [27]:
def point_inside_polygon(x, y, poly):
    n = len(poly)
    inside = False
    p1x, p1y = poly[0]
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside

In [32]:
def fill_polygon(grid, polygon):
    for row_index, row in enumerate(grid):
        for col_index, col in enumerate(row):
            value = grid[row_index][col_index]
            if point_inside_polygon(row_index, col_index, polygon) and value != "#":
                grid[row_index][col_index] = "0"
    return grid 

In [36]:
# polygon = list(steps_dic.keys())
# filled_grid = fill_polygon(grid, polygon)
# display_grid(filled_grid)

# # count how many tiles are enclosed by the loop/polygon
# count = 0
# for row_index, row in enumerate(grid):
#     for col_index, col in enumerate(row):
#         value = grid[row_index][col_index]
#         if value == "0":
#             count += 1
# print(f"\n{count} tiles are enclosed by the loop")

In [70]:
def generate_grid2(lines, n_rows, n_cols, print_grid=False):
    grid = [[' ' for _ in range(n_cols)] for _ in range(n_rows)]
    for i, row in enumerate(lines):
        print(i)
        for j, element in enumerate(row[0]):
            print(j)
            print(i, j, grid[i][j], element)
            grid[i][j] = element
    if print_grid==True:
        display_grid(grid)
    return grid

In [95]:
with open("test5_input.txt", "r") as file:
    lines = file.readlines() 

# remove newline text from each line
lines = [line.rstrip("\n").split(" ") for line in lines]
grid = [list(line[0]) for line in lines]

# get metadata
n_rows = len(grid)
n_cols = len(grid[0])
print(f"Shape: {n_cols} columns, {n_rows} rows")
display_grid(lines)

Shape: 11 columns, 9 rows
...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........


In [97]:
start_pos

(1, 1)

In [96]:
start_pos = next_position = find_start(grid)
solutions = find_paths_recursive(grid, current_path=[start_pos])
path_values = compute_cell_values(grid, solutions)

IndexError: list index out of range

In [85]:
def find_start(grid):
    for i, row in enumerate(grid):
        for j, value in enumerate(row):
            if value == "S":
                start_pos = (i, j)
                return start_pos

# example
find_start(grid)

(1, 1)

In [86]:
len(grid[0])

11

In [97]:
def find_paths_recursive(grid, current_path=[(0,0)], solutions=[]):

    n = len(grid)
    m = len(grid[0])
    last_cell = current_path[-1]

    dirs = {
        "down": [1, 0],
        "up": [-1, 0],
        "right": [0, 1],
        "left": [0, -1]
    }
    possible_values_dic = {
        "down": ["|", "L", "J"],
        "up": ["|", "F", "7"],
        "right": ["-", "J", "7"],
        "left": ["-", "L", "F"]
    }
    
    for dir, offset in dirs.items():
        x, y = offset
        new_i = last_cell[0] + x
        new_j = last_cell[1] + y
        new_cell = (new_i, new_j)
        # print("Try", new_cell)

        # get possible values
        possible_values = possible_values_dic[dir]
        
        # Check if new cell is in grid
        if 0 > new_i or new_i >= n or 0 > new_j or new_j >= m:
            # print(">>> out of index")
            continue

        # Check if new cell is already in path
        if new_cell in current_path[1:]:
            # print(">>> already in path")
            continue

        # get new value
        new_val = grid[new_i][new_j]

        # Check if valid cell
        if new_val not in possible_values:
            # print(f">>> Non-valid value: {new_val}")
            continue

        # print(">>> Valid")

        # Add cell to current path
        current_path_copy = current_path.copy()
        current_path_copy.append(new_cell)
        # print("Path:", current_path_copy)

        if new_i==n-1 and new_j == n-1:
            solutions.append(current_path_copy)

        solutions.append(current_path_copy)

        # Create new current_path array for every direction
        find_paths_recursive(grid, current_path_copy, solutions)
        
    return solutions

def compute_cell_values(grid1, solutions):
    path_values = []
    for solution in solutions:
        solution_values = []
        for cell in solution:
            solution_values.append(grid1[cell[0]][cell[1]])
        path_values.append(solution_values)
    return path_values

lines, n_rows, n_cols = get_data("test5_input.txt")
grid = generate_grid2(lines, n_rows, n_cols, print_grid=True)

start_pos = next_position = find_start(grid)

solutions = find_paths_recursive(grid, current_path=[start_pos])
path_values = compute_cell_values(grid, solutions)

# print('Solutions:')
# print(solutions)
# print('Values:')
# print(path_values)


Shape: 11 columns, 9 rows
...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........


In [98]:
steps_dic = {start_pos: 0}
for path in solutions:
    start = path[0]
    end = path[-1]
    steps = len(path) - 1
    if end in steps_dic.keys():
        if steps < steps_dic[end]:
            steps_dic[end] = steps
    else:
        steps_dic[end] = steps
loop_path = list(steps_dic.keys())
steps_dic

{(1, 1): 0,
 (2, 1): 1,
 (3, 1): 2,
 (4, 1): 3,
 (5, 1): 4,
 (6, 1): 5,
 (7, 1): 6,
 (7, 2): 7,
 (7, 3): 8,
 (7, 4): 9,
 (6, 4): 10,
 (5, 4): 11,
 (5, 3): 12,
 (5, 2): 13,
 (4, 2): 14,
 (3, 2): 15,
 (2, 2): 16,
 (2, 3): 17,
 (2, 4): 18,
 (2, 5): 19,
 (2, 6): 20,
 (2, 7): 21,
 (2, 8): 22,
 (3, 8): 23,
 (4, 8): 22,
 (5, 8): 21,
 (5, 7): 20,
 (5, 6): 19,
 (6, 6): 18,
 (7, 6): 17,
 (7, 7): 16,
 (7, 8): 15,
 (7, 9): 14,
 (6, 9): 13,
 (5, 9): 12,
 (4, 9): 11,
 (3, 9): 10,
 (2, 9): 9,
 (1, 9): 8,
 (1, 8): 7,
 (1, 7): 6,
 (1, 6): 5,
 (1, 5): 4,
 (1, 4): 3,
 (1, 3): 2,
 (1, 2): 1}

In [102]:
for pos in steps_dic.keys():
    grid[pos[0]][pos[1]]="#"
display_grid(grid)

...........
.#########.
.#########.
.##.....##.
.##.....##.
.####.####.
.#..#.#..#.
.####.####.
...........


In [134]:
outside_cells = []
for i, row in enumerate(grid):
    for j, col in enumerate(row):
        if grid[i][j] == "#":
            # break
            pass
        else:
            grid[i][j] = str("0")
display_grid(grid)

00000000000
0#########0
0#########0
0##00000##0
0##00000##0
0####0####0
0#00#0#00#0
0####0####0
00000000000


In [135]:
grid

[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '#', '#', '#', '#', '#', '#', '#', '#', '#', '0'],
 ['0', '#', '#', '#', '#', '#', '#', '#', '#', '#', '0'],
 ['0', '#', '#', '0', '0', '0', '0', '0', '#', '#', '0'],
 ['0', '#', '#', '0', '0', '0', '0', '0', '#', '#', '0'],
 ['0', '#', '#', '#', '#', '0', '#', '#', '#', '#', '0'],
 ['0', '#', '0', '0', '#', '0', '#', '0', '0', '#', '0'],
 ['0', '#', '#', '#', '#', '0', '#', '#', '#', '#', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]